In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, StackingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score, silhouette_score, adjusted_rand_score, normalized_mutual_info_score, rand_score, make_scorer, calinski_harabasz_score
import pickle

In [ ]:
data = pd.read_csv("D:\RGR\rgr\pages\card_transdata.csv")

In [ ]:
data = data[:5000]

In [ ]:
data.to_csv('card_transdata_upd.csv', encoding='utf-8   ')

NameError: name 'data' is not defined

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      5000 non-null   int64  
 1   distance_from_home              5000 non-null   float64
 2   distance_from_last_transaction  5000 non-null   float64
 3   ratio_to_median_purchase_price  5000 non-null   float64
 4   repeat_retailer                 5000 non-null   bool   
 5   used_chip                       5000 non-null   bool   
 6   used_pin_number                 5000 non-null   bool   
 7   online_order                    5000 non-null   bool   
 8   fraud                           5000 non-null   bool   
dtypes: bool(5), float64(3), int64(1)
memory usage: 180.8 KB


In [ ]:
data

,Unnamed: 0,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,0,57.877857,0.311140,1.945940,True,True,False,False,False
1,1,10.829943,0.175592,1.294219,True,False,False,False,False
2,2,5.091079,0.805153,0.427715,True,False,False,True,False
3,3,2.247564,5.600044,0.362663,True,True,False,True,False
4,4,44.190936,0.566486,2.222767,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...
4995,4995,10.285723,1.559981,0.674788,True,True,False,False,False
4996,4996,8.599866,0.113014,1.739850,True,False,False,True,False
4997,4997,3.420348,0.117740,0.153573,True,True,False,False,False
4998,4998,2.100580,0.275625,19.331504,True,False,False,True,True


In [ ]:
data = data.drop(columns=['Unnamed: 0'])


In [ ]:
y = data["fraud"]
X = data.drop(["fraud"], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

In [ ]:
model1 = svm.SVC(kernel='linear')  # Линейное ядро
model1.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
with open('SVM.pkl', 'wb') as pickle_out:
    pickle.dump(model1, pickle_out)

In [ ]:
with open('SVM.pkl', 'rb') as pickle_in:
    unpickled_list = pickle.load(pickle_in)

In [ ]:
model2 = KMeans(n_clusters=3, random_state=42, n_init='auto')
model2.fit(X_train)

KMeans(n_clusters=3, n_init='auto', random_state=42)

In [ ]:
with open('kmeans.pkl', 'wb') as pickle_out:
    pickle.dump(model2, pickle_out)

In [ ]:
model3 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

model3.fit(X_train, y_train)


GradientBoostingClassifier(random_state=42)

In [ ]:
with open('GradientBoosting.pkl', 'wb') as pickle_out:
    pickle.dump(model3, pickle_out)

In [ ]:
model4 = BaggingClassifier(n_estimators=10, random_state=42)

model4.fit(X_train, y_train)


BaggingClassifier(random_state=42)

In [ ]:
with open('Bagging.pkl', 'wb') as pickle_out:
    pickle.dump(model4, pickle_out)

In [ ]:
base_classifiers = [
    ('rf', RandomForestClassifier()),
    ('KNN', KNeighborsClassifier()),
    ('SVM',svm.SVC())
]
meta_classifier = LogisticRegression(random_state=42)

model5 = StackingClassifier(estimators=base_classifiers, final_estimator=meta_classifier)
model5.fit(X_train, y_train)


StackingClassifier(estimators=[('rf', RandomForestClassifier()),
                               ('KNN', KNeighborsClassifier()),
                               ('SVM', SVC())],
                   final_estimator=LogisticRegression(random_state=42))

In [ ]:
with open('Stacking.pkl', 'wb') as pickle_out:
    pickle.dump(model5, pickle_out)

In [ ]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [ ]:
model6 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(1, activation='sigmoid')

])

model6.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model6.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))




Epoch 1/50


125/125 [==============================] - 2s 6ms/step - loss: 0.6723 - accuracy: 0.8593 - val_loss: 0.4422 - val_accuracy: 0.9150
Epoch 2/50
125/125 [==============================] - 0s 3ms/step - loss: 0.3041 - accuracy: 0.9197 - val_loss: 0.2920 - val_accuracy: 0.9200
Epoch 3/50
125/125 [==============================] - 0s 4ms/step - loss: 0.2665 - accuracy: 0.9245 - val_loss: 0.2208 - val_accuracy: 0.9230
Epoch 4/50
125/125 [==============================] - 0s 3ms/step - loss: 0.2174 - accuracy: 0.9290 - val_loss: 0.3173 - val_accuracy: 0.9210
Epoch 5/50
125/125 [==============================] - 0s 3ms/step - loss: 0.1957 - accuracy: 0.9285 - val_loss: 0.2147 - val_accuracy: 0.9290
Epoch 6/50
125/125 [==============================] - 0s 3ms/step - loss: 0.1822 - accuracy: 0.9340 - val_loss: 0.1909 - val_accuracy: 0.9260
Epoch 7/50
125/125 [==============================] - 0s 3ms/step - loss: 0.1739 - accuracy: 0.9385 - val_loss: 0.2058 - val_accuracy: 0.9290
Ep

In [ ]:
model6.save('Neural.h5')


c:\Users\Dmitry\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
